(model_deployment)=
# Model deployment

One of the main goals of PyMC-Marketing is to facilitate the deployment of its models. 

This is achieved by building our models on top of [ModelBuilder](https://www.pymc-marketing.io/en/stable/api/generated/pymc_marketing.model_builder.ModelBuilder.html#pymc_marketing.model_builder.ModelBuilder) that offers a scikit-learn-like API and makes PyMC models easy to deploy.

PyMC-marketing models inherit 2 easy-to-use methods: `save` and `load` that can be used after the model has been fitted. All models can be configured with two standard dictionaries: `model_config` and `sampler_config` that are serialized during `save` and persisted after `load`, allowing model reuse across workflows.

We will illustrate this functionality with the example model described in the [MMM Example Notebook](https://www.pymc-marketing.io/en/stable/notebooks/mmm/mmm_example.html). For sake of generality, we ommit most technical details here.

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from pymc_marketing.prior import Prior

az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100

%config InlineBackend.figure_format = "retina"

In [ ]:
seed = sum(map(ord, "mmm"))
rng = np.random.default_rng(seed=seed)

Let's load the dataset:

In [ ]:
url = "https://raw.githubusercontent.com/pymc-labs/pymc-marketing/main/data/mmm_example.csv"
df = pd.read_csv(url, parse_dates=["date_week"])

columns_to_keep = [
    "date_week",
    "y",
    "x1",
    "x2",
    "event_1",
    "event_2",
    "dayofyear",
]

data = df[columns_to_keep].copy()
data["t"] = np.arange(df.shape[0])
data.head()

But for our model we need much smaller dataset, many of the previous features were contributing to generation of others, now as our target variable is computed we can filter out not needed columns:

## Model and sampling configuration

### Model configuration

We first illustrate the use of `model_config` to define custom priors within the model.

Because there are potentially many variables that can be configured, each model provides a `default_model_config` attribute. This will allow you to see which settings are available by default and only define the ones you need to change.

We need to create a dummy model to be able to see the configuration dictionary.

In [ ]:
adstock = GeometricAdstock(l_max=8)
saturation = LogisticSaturation()

dummy_model = MMM(
    date_column="date_week",
    channel_columns=["x1", "x2"],
    adstock=adstock,
    saturation=saturation,
    control_columns=[
        "event_1",
        "event_2",
        "t",
    ],
    yearly_seasonality=2,
)
dummy_model.default_model_config

We can change the parameters that go into the distribution of each term.
In this case we'll just simply replace the `sigma` for `saturation_beta` with a custom one:

In [ ]:
n_channels = 2

total_spend_per_channel = data[["x1", "x2"]].sum(axis=0)
spend_share = total_spend_per_channel / total_spend_per_channel.sum()

# The scale necessary to make a HalfNormal distribution have unit variance
HALFNORMAL_SCALE = 1 / np.sqrt(1 - 2 / np.pi)
prior_sigma = HALFNORMAL_SCALE * n_channels * spend_share.to_numpy()
prior_sigma

In [ ]:
saturation_beta = Prior("HalfNormal", sigma=prior_sigma, dims="channel")
my_model_config = {"saturation_beta": saturation_beta}

In [ ]:
my_model_config

As mentioned in the original notebook: _"For the prior specification there is no right or wrong answer. It all depends on the data, the context and the assumptions you are willing to make. It is always recommended to do some prior predictive sampling and sensitivity analysis to check the impact of the priors on the posterior. We skip this here for the sake of simplicity. If you are not sure about specific priors, the `MMM` class has some default priors that you can use as a starting point."_

### Sampling  configuration

The second feature we can customize is `sampler_config`. Similar to `model_config`, it's a dictionary that gets saved and contains things you would usually pass to the `fit()` kwargs. It's not mandatory to create your own `sampler_config`. The default `MMM.sampler_config` is empty because the default sampling parameters usually prove sufficient for a start.

In [ ]:
dummy_model.default_sampler_config

In [ ]:
my_sampler_config = {
    "tune": 1000,
    "draws": 1000,
    "chains": 4,
    "target_accept": 0.91,
    "nuts_sampler": "numpyro",
}

Let's finally assemble our model!

In [ ]:
mmm = MMM(
    model_config=my_model_config,
    sampler_config=my_sampler_config,
    date_column="date_week",
    channel_columns=["x1", "x2"],
    adstock=adstock,
    saturation=saturation,
    control_columns=[
        "event_1",
        "event_2",
        "t",
    ],
    yearly_seasonality=2,
)

We can confirm our settings are being used

In [ ]:
mmm.model_config["saturation_beta"]

In [ ]:
mmm.sampler_config

## Model Fitting

Note that we didn't pass the dataset to the class constructor itself. This is done to mimick the `scikit-learn` API, and make it easier to get started on PyMC-Marketing models.

In [ ]:
# Split X, and y
X = data.drop("y", axis=1)
y = data["y"]

All that's left now is to finally fit the model:

As you can see below, you can still pass the sampler kwargs directly to `fit()` method. However, only those kwargs passed using `sampler_config` will be saved and reused after loading the model.

In [ ]:
mmm.fit(X=X, y=y, random_seed=rng)

The `fit()` method automatically builds the model using the priors from `model_config`, and assigns the created model to our instance. You can access it as a normal attribute.

In [ ]:
type(mmm.model)

In [ ]:
mmm.graphviz()

posterior trace can be accessed by `fit_result` attribute

In [ ]:
mmm.fit_result

If you wish to inspect the entire inference data, use the `idata` attribute. Within `idata`, you can find the entire dataset passed to the model under `fit_data`.

In [ ]:
mmm.idata

## Saving and loading a fitted model

All the data passed to the model on initialization is stored in `idata.attrs`. This will be used later in the `save()` method to convert both this data and all the fit data into the netCDF format. You can read more about this format [here](https://python.arviz.org/en/stable/getting_started/XarrayforArviZ.html).

The `save` and `load` method only require a path to inform where the model should be saved and loaded from.

In [ ]:
mmm.save("my_saved_model.nc")

In [ ]:
loaded_model = MMM.load("my_saved_model.nc")

In [ ]:
loaded_model.model_config["saturation_beta"]

In [ ]:
loaded_model.idata.attrs

In [ ]:
loaded_model.graphviz()

In [ ]:
loaded_model.idata

A loaded model is ready to be used for sampling and prediction, making use of the previous fitting results and data if needed.

In [ ]:
loaded_model.sample_posterior_predictive(
    X, extend_idata=True, combined=False, random_seed=rng
)

In [ ]:
az.plot_ppc(loaded_model.idata);

## Other models

Even though this introduction is using `MMM`, all other PyMC-Marketing models (MMM and CLV) provide these functionalities as well.

## Summary

The PyMC-Marketing functionalities described here are intended to facilitate model sharing among data science teams without demanding extensive modelling technical knowledge for everyone involved. We are still iterating on our API and would love to hear more feedback from our users!

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w -p pytensor